In [ ]:
from azureml.core import Workspace, Experiment, RunConfiguration
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.pipeline.core import Pipeline, PipelineData, StepSequence
from azureml.pipeline.steps import PythonScriptStep
import json

In [ ]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [ ]:
# load experiment cfg
with open("experiment_cfg.json", "r") as cfg_file:
    cfg = json.load(cfg_file)

## Deployment Step Script

In [ ]:
%%writefile ./src/deploy.py

from azureml.core import Run, Model
import os
import pandas as pd
import joblib
from argparse import ArgumentParser

run = Run.get_context()
ws = run.experiment.workspace

for child in run.parent.get_children():
    if child.name == 'preprocess.py':
        child.register_model('SKLearnPreprocessPipeline', 'outputs/pipelines.pkl')
    elif child.name == 'train.py':
        child.register_model('SKLearnClassifier', 'outputs/model.pkl')
        child.register_model('SKLearnRegressor', 'outputs/model_regressor.pkl')
        
inference_config = InferenceConfig(entry_script="score.py", environment=ws.environments['WILO_POC_sklearn'], source_directory='.')

aksconfig = AksWebservice.deploy_configuration()

service = Model.deploy(workspace=ws, 
                           name='SKLearnWebservice',
                           models= [ws.models['SKLearnPreprocessPipeline'], ws.models['SKLearnClassifier'], ws.models['SKLearnRegressor']],
                           inference_config=inference_config, 
                           deployment_config=aksconfig,
                           deployment_target=ws.compute_targets[cfg['compute_target']])

service.wait_for_deployment()
        
run.complete()

## Create Pipeline

### Create RunConfig with CondaEnv

In [1]:
# create run config
run_config = RunConfiguration()
run_config.environment = ws.environments[cfg['env_name']]

NameError: name 'RunConfiguration' is not defined

### Define PipelineData Objects

In [2]:
# input data
symptoms_data = ws.datasets['symptomcodes.csv'].as_named_input('symptomcodes')
raw_input_data = ws.datasets['ItemResourceData.csv'].as_named_input('df_raw')

# prepared data
prepared_data = PipelineData("prepared_data", datastore=ws.datastores['workspaceblobstore'], is_directory=True)

# preprocessed data
preprocessed_data = PipelineData("preprocessed_data", datastore=ws.datastores['workspaceblobstore'], is_directory=True)

# # output
# pipeline_data = PipelineData("pipeline_data", datastore=ws.datastores['workspaceblobstore'], is_directory=False)
# trained_classifier = PipelineData("trained_classifier", datastore=ws.datastores['workspaceblobstore'], is_directory=False)
# trained_regressor = PipelineData("trained_regressor", datastore=ws.datastores['workspaceblobstore'], is_directory=False)

NameError: name 'ws' is not defined

### Define Pipeline Steps

In [ ]:
prepare_step = PythonScriptStep(script_name='prepare.py', source_directory='src',
                                inputs=[symptoms_data, raw_input_data], outputs=[prepared_data],
                                arguments=['--output', prepared_data],
                                compute_target=ws.compute_targets['mlcompute'], runconfig=run_config, allow_reuse=True)

In [ ]:
preprocess_step = PythonScriptStep(script_name='preprocess.py', source_directory='src',
                                inputs=[prepared_data], outputs=[preprocessed_data],
                                arguments=['--input', prepared_data, '--output', preprocessed_data],
                                compute_target=ws.compute_targets['mlcompute'], runconfig=run_config, allow_reuse=True)

In [ ]:
train_step = PythonScriptStep(script_name='train.py', source_directory='src',
                                inputs=[preprocessed_data], arguments=['--input', preprocessed_data],
                                compute_target=ws.compute_targets['mlcompute'], runconfig=run_config, allow_reuse=True)

In [ ]:
deploy_step = PythonScriptStep(script_name='deploy.py', source_directory='src',
                               compute_target=ws.compute_targets['mlcompute'], runconfig=run_config, allow_reuse=True)

### Create Pipeline

In [ ]:
pipeline = Pipeline(ws, StepSequence([[prepare_step, preprocess_step, train_step], deploy_step]))

## Run Experiment

In [ ]:
exp = Experiment(ws, cfg['experiment_name'])
run = exp.submit(pipeline)
run.wait_for_completion(show_output=True)